# Resizing an image in FPGA - Deployment Flow
This reference design illustrates how to run a resizer IP on the FPGA(PL) using Jupyter Notebooks and Python

## Block diagram:

![](./images/resize_bd.png)

## Contents    
  

* [Resizing an image in Programmable Logic](#Resizing-an-image-in-Programmable-Logic)
    * [Import libraries](#Import-libraries)
    * [Download the Resize IP bitstream](#Download-the-Resize-IP-bitstream)
    * [Create an Image object using PIL in SW](#Create-an-Image-object-using-PIL-in-SW)
    * [Display the image to be resized](#Display-the-image-to-be-resized)
    * [Resizing in Software](#Resizing-in-Software)
    * [Resize in Programmable Logic](#Resize-in-Programmable-Logic)
* [References](#References)

## Import libraries

In [ ]:
from PIL import Image
import numpy as np
from IPython.display import display, Markdown
from pynq import Xlnk
from pynq import Overlay


## Download the Resize IP bitstream

In [ ]:
resize_design = Overlay("/opt/python3.6/lib/python3.6/site-packages/pynq/overlays/resize/resize.bit")

#### Create DMA and Resizer IP objects

In [ ]:
dma = resize_design.axi_dma_0
resizer = resize_design.resize_accel_0

## Create an Image object using PIL in SW
#### Load image from the SD card and create an Image object

In [ ]:
image_path = "./images/paris.jpg"
original_image = Image.open(image_path)
original_image.load()

#### Create a numpy array of the pixels

In [ ]:
input_array = np.array(original_image)

[Contents](#Contents)

## Display the image to be resized

In [ ]:
input_image = Image.fromarray(input_array)
# display(input_image)

# def printmd(string):
#     display(Markdown('<h1 style="color:DeepPink"> {}</h1>'.format(string)))

#### Original image size

In [ ]:
old_width, old_height = original_image.size
# printmd("Image size: {}x{} pixels.".format(old_width, old_height))

[Contents](#Contents)

## Resizing in Software

#### Setting image resize dimensions
**Note:** Downscale factor range:  1 to 7 (by design of the resize IP)

In [ ]:
resize_factor = 2
new_width, new_height = int(old_width/resize_factor), int(old_height/resize_factor)

#### Timing in SW

In [ ]:
# %%timeit
# resized_image_sw = original_image.resize((new_width, new_height), Image.BILINEAR)

#### Using resize() method from the PIL library

In [ ]:
resized_image_sw = original_image.resize((new_width, new_height), Image.BILINEAR)

[Contents](#Contents)

#### Display resized image

In [ ]:
output_array_sw = np.array(resized_image_sw)
result_sw = Image.fromarray(output_array_sw)
# display(result_sw)

#### Resized image size

In [ ]:
width_sw, height_sw = resized_image_sw.size
# printmd("Image size resized in SW: {}x{} pixels.".format(width_sw, height_sw))

[Contents](#Contents)

## Resize in Programmable Logic

#### Allocating memory to process data on PL
Data is provided through contiguous memory locations.

The size of the buffer depends on the size of the input or output data.
The image dimensions extracted from the read image are used to allocate contiguous memory as follows.
We will use `cma_array` of the corresponding size.

In [ ]:
xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(old_height, old_width, 3), dtype=np.uint8)
out_buffer = xlnk.cma_array(shape=(new_height, new_width, 3), dtype=np.uint8)

__Note: In the following example, we are only dealing with one image. We will just send one image to the kernel and obtain the results.__

__Also Note: The `input_array` has to be copied into the contiguous memory array(deep copy).__

[Contents](#Contents)

#### Display the image in buffer

In [ ]:
in_buffer[:] = input_array
buf_image = Image.fromarray(in_buffer)
# display(buf_image)
# printmd("Image size: {}x{} pixels.".format(old_width, old_height))


#### Run the Resizer IP
Now we will push the data from input buffer through the pipeline to the output buffer.   
Providing scalar inputs and running the kernel

In [ ]:
resizer.write(0x10, old_height) # src rows
resizer.write(0x18, old_width)  # src cols
resizer.write(0x20, new_height) # dst rows
resizer.write(0x28, new_width)  # dst cols

def run_kernel():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    resizer.write(0x00,0x81)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

run_kernel()

result = Image.fromarray(out_buffer)
# display(result)
# printmd("Resized in Hardware(PL): {}x{} pixels.".format(new_width, new_height))

#### Timing in HW

In [ ]:
# %%timeit

# resizer.write(0x10, old_height) # src rows
# resizer.write(0x18, old_width)  # src cols
# resizer.write(0x20, new_height) # dst rows
# resizer.write(0x28, new_width)  # dst cols

# def run_kernel():
#     dma.sendchannel.transfer(in_buffer)
#     dma.recvchannel.transfer(out_buffer)
#     resizer.write(0x00,0x81)
#     dma.sendchannel.wait()
#     dma.recvchannel.wait()

# run_kernel()

#### Deploy

In [ ]:
result = Image.fromarray(out_buffer)
result.save('./paris_resized.jpg')

[Contents](#Contents)

#### Reset Xlnk

In [ ]:
xlnk.xlnk_reset()

# References

https://pillow.readthedocs.io/en/3.1.x/index.html   
https://github.com/Xilinx/PYNQ/blob/master/docs/source/python_environment.ipynb   
https://github.com/Xilinx/PYNQ/blob/master/docs/source/jupyter_notebooks.ipynb   
https://github.com/Xilinx/PYNQ/blob/master/docs/source/jupyter_notebooks_advanced_features.ipynb   